In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE41575"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE41575"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE41575.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE41575.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE41575.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Role of platelet micrornas in sickle cell disease"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: Meg-01 cells'], 1: ['genotype/variation: miR1225 overexpression', 'genotype/variation: control']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Availability
# This seems to be a study focusing on microRNAs in sickle cell disease
# The description mentions "Role of platelet micrornas in sickle cell disease"
# This suggests it's primarily miRNA data, not gene expression
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# {0: ['cell line: Meg-01 cells'], 1: ['genotype/variation: miR1225 overexpression', 'genotype/variation: control']}

# For trait (Sickle Cell Anemia):
# The data doesn't explicitly show sickle cell status for samples
# It mentions cell lines and miRNA overexpression vs control
# Row 1 has "genotype/variation" which could be related to the trait
trait_row = 1

# For age:
# No information about age is provided in the sample characteristics
age_row = None

# For gender:
# No information about gender is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert the trait value to binary format.
    For sickle cell anemia, we'll use the genotype/variation field.
    """
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the provided data, we need to interpret genotype/variation values
    # miR1225 overexpression vs control could indicate an experimental setup
    # rather than disease status
    if 'control' in value.lower():
        return 0  # Control
    elif 'overexpression' in value.lower():
        return 1  # Experimental condition
    
    return None  # For any other values

def convert_age(value):
    """
    Convert age value to continuous format.
    """
    # Not used as age_row is None
    return None

def convert_gender(value):
    """
    Convert gender value to binary format.
    """
    # Not used as gender_row is None
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None and 'clinical_data' in locals():
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1019586': [1.0], 'GSM1019587': [1.0], 'GSM1019588': [1.0], 'GSM1019589': [1.0], 'GSM1019590': [1.0], 'GSM1019591': [0.0], 'GSM1019592': [0.0], 'GSM1019593': [0.0], 'GSM1019594': [0.0], 'GSM1019595': [0.0]}
Clinical data saved to ../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE41575.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM1019586"	"GSM1019587"	"GSM1019588"	"GSM1019589"	"GSM1019590"	"GSM1019591"	"GSM1019592"	"GSM1019593"	"GSM1019594"	"GSM1019595"
First data line: "A_23_P100001"	11.38730201	11.93312075	11.70949679	11.91858179	12.34590109	13.25895613	12.94793603	12.90752861	12.90612796	12.907528
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown appear to be Agilent microarray probe IDs (e.g., "A_23_P100001")
# These are not standard human gene symbols (which would look like "BRCA1", "TP53", etc.)
# These probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE41575
Line 6: !Series_title = Role of platelet micrornas in sickle cell disease
Line 7: !Series_geo_accession = GSE41575
Line 8: !Series_status = Public on Jun 20 2013
Line 9: !Series_submission_date = Oct 15 2012
Line 10: !Series_last_update_date = Oct 11 2016
Line 11: !Series_pubmed_id = 23593351
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_type = Non-coding RNA profiling by array
Line 16: !Series_sample_id = GSM1019586
Line 17: !Series_sample_id = GSM1019587
Line 18: !Series_sample_id = GSM1019588
Line 19: !Series_sample_id = GSM1

### Step 6: Gene Identifier Mapping

In [7]:
# Let's implement a more robust approach to extract gene annotation from the SOFT file
print("Extracting gene annotation from SOFT file...")

try:
    # Approach 1: Use get_gene_annotation function if it works
    gene_annotation = get_gene_annotation(soft_file)
    
    if gene_annotation is None or len(gene_annotation) == 0:
        # If that fails, use a more manual approach
        print("Standard approach failed, trying alternative method to extract gene annotation")
        
        # Read the SOFT file and extract the platform table
        platform_data = []
        with gzip.open(soft_file, 'rt') as file:
            in_platform_section = False
            for line in file:
                if line.startswith('^PLATFORM'):
                    in_platform_section = True
                if in_platform_section and line.startswith('!platform_table_begin'):
                    # Next line should be the header
                    header = next(file).strip()
                    platform_data.append(header)
                    # Read until the end of the platform table
                    for table_line in file:
                        if table_line.startswith('!platform_table_end'):
                            break
                        platform_data.append(table_line.strip())
                    break
        
        if platform_data:
            import io
            platform_text = '\n'.join(platform_data)
            gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                        low_memory=False, on_bad_lines='skip')
            
            print(f"Successfully extracted gene annotation with {len(gene_annotation)} rows")
            print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
        else:
            print("Could not find platform table in SOFT file")
            gene_annotation = None
    else:
        print(f"Successfully extracted gene annotation with standard method: {len(gene_annotation)} rows")
        print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")

    # If we have gene annotation data, check for ID and gene symbol columns
    if gene_annotation is not None and len(gene_annotation) > 0:
        # Look for columns related to probe ID
        id_cols = [col for col in gene_annotation.columns 
                  if 'id' in col.lower() or 'probe' in col.lower() or col == 'ID']
        
        # Look for columns related to gene symbols
        gene_symbol_cols = [col for col in gene_annotation.columns 
                          if 'symbol' in col.lower() or 'gene' in col.lower()]
        
        print(f"Potential ID columns: {id_cols}")
        print(f"Potential gene symbol columns: {gene_symbol_cols}")
        
        # Check if we have sample values in these columns
        if id_cols:
            print(f"\nSample values from ID column '{id_cols[0]}':")
            print(gene_annotation[id_cols[0]].head())
        
        if gene_symbol_cols:
            print(f"\nSample values from gene symbol column '{gene_symbol_cols[0]}':")
            print(gene_annotation[gene_symbol_cols[0]].head())
        
        # Determine which columns to use for mapping
        id_col = id_cols[0] if id_cols else None
        gene_symbol_col = gene_symbol_cols[0] if gene_symbol_cols else None
        
        # Create the mapping dataframe
        if id_col and gene_symbol_col:
            print(f"\nCreating gene mapping using columns: {id_col} -> {gene_symbol_col}")
            mapping_data = get_gene_mapping(gene_annotation, id_col, gene_symbol_col)
            print(f"Created mapping with {len(mapping_data)} rows")
            
            # Apply the mapping to convert probe-level measurements to gene expression
            gene_data = apply_gene_mapping(gene_data, mapping_data)
            print(f"Successfully mapped probes to genes. Gene expression data has {len(gene_data)} genes.")
            
            # Set gene_available to True since we successfully processed gene expression data
            is_gene_available = True
        else:
            print("\nCould not identify appropriate ID and gene symbol columns")
            print("Unable to create gene mapping")
            gene_data = None
            is_gene_available = False
    else:
        print("\nNo gene annotation data available")
        gene_data = None
        is_gene_available = False
        
except Exception as e:
    print(f"Error processing gene annotation: {e}")
    gene_data = None
    is_gene_available = False

# Update cohort info with the correct gene availability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If gene data was successfully mapped, print a preview
if gene_data is not None:
    print("\nPreview of mapped gene expression data:")
    print(gene_data.head())
else:
    print("\nNo gene expression data available for preview")

Extracting gene annotation from SOFT file...


Successfully extracted gene annotation with standard method: 334259 rows
Columns in gene annotation: ['ID', 'ControlType', 'miRNA_ID', 'SPOT_ID']
Potential ID columns: ['ID', 'miRNA_ID', 'SPOT_ID']
Potential gene symbol columns: []

Sample values from ID column 'ID':
0       bkv-miR-B1-3p
1       bkv-miR-B1-5p
2    ebv-miR-BART1-3p
3    ebv-miR-BART1-5p
4      ebv-miR-BART10
Name: ID, dtype: object

Could not identify appropriate ID and gene symbol columns
Unable to create gene mapping

No gene expression data available for preview
